In [1]:
from browser_use import Agent
from dotenv import load_dotenv
load_dotenv()

import asyncio
import nest_asyncio

nest_asyncio.apply()



INFO     [browser_use] BrowserUse logging setup complete with level info
INFO     [root] Anonymized telemetry enabled. See https://docs.browser-use.com/development/telemetry for more information.


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", #gemini-2.5-pro-exp-03-25
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [4]:
# llm = ChatGoogleGenerativeAI(
#     model="gemini-2.5-pro-exp-03-25", #gemini-2.5-pro-exp-03-25
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
#     # other params...
# )

In [3]:
from browser_use import Agent
import asyncio
from browser_use import Browser
from browser_use import BrowserConfig

# Basic configuration
config = BrowserConfig(
    headless=True,
    disable_security=True
)

browser = Browser(config=config)



async def main():
    agent = Agent(
        task="Compare the price of gpt-4o and DeepSeek-V3",
        llm=llm,
        browser=browser,
        retry_delay=20,
        max_actions_per_step=15
    )
    result = await agent.run()
    return result


In [4]:
sensitive_data = {'first_name': 'magnus', 'last_name': 'Carlson', 'email': 'abc@cb.com', "phone": "1255532", 'available_from': "April 5, 2025", 
                 'desired_salary': "20k $", "resume_path": "/home/bhanu/projects/ai-playground/browser/tmp.txt", "years_of_experience": "5"}
initial_actions = [
	{'open_tab': {'url': 'http://localhost:8001/static/index.html'}},
]



In [6]:
import os
import sys
from pathlib import Path

from browser_use.agent.views import ActionResult

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("auto_apply.ipynb"))))
import asyncio
import logging


from browser_use import Agent, Controller
from browser_use.browser.browser import Browser, BrowserConfig
from browser_use.browser.context import BrowserContext

logger = logging.getLogger(__name__)

controller = Controller()


@controller.action(
	'Upload file to interactive element with file path ',
)
async def upload_file(index: int, path: str, browser: BrowserContext, available_file_paths: list[str]):
	if path not in available_file_paths:
		return ActionResult(error=f'File path {path} is not available')

	if not os.path.exists(path):
		return ActionResult(error=f'File {path} does not exist')

	dom_el = await browser.get_dom_element_by_index(index)

	file_upload_dom_el = dom_el.get_file_upload_element()

	if file_upload_dom_el is None:
		msg = f'No file upload element found at index {index}'
		logger.info(msg)
		return ActionResult(error=msg)

	file_upload_el = await browser.get_locate_element(file_upload_dom_el)

	if file_upload_el is None:
		msg = f'No file upload element found at index {index}'
		logger.info(msg)
		return ActionResult(error=msg)

	try:
		await file_upload_el.set_input_files(path)
		msg = f'Successfully uploaded file to index {index}'
		logger.info(msg)
		return ActionResult(extracted_content=msg, include_in_memory=True)
	except Exception as e:
		msg = f'Failed to upload file to index {index}: {str(e)}'
		logger.info(msg)
		return ActionResult(error=msg)


In [10]:
from browser_use import Agent
import asyncio
from browser_use import Browser
from browser_use import BrowserConfig
from browser_use import Controller, ActionResult
# Initialize the controller
# controller = Controller()

@controller.action('Ask human in case you need anything')
def ask_human(question: str) -> str:
    answer = input(f'\n{question}\nInput: ')
    return ActionResult(extracted_content=answer)

from browser_use import Agent, SystemPrompt

class MySystemPrompt(SystemPrompt):
    def important_rules(self) -> str:
        # Get existing rules from parent class
        existing_rules = super().important_rules()

        # Add your custom rules
        new_rules = """
** IMPORTANT RULE **:
- Fill the fields dynamically, if you can't set a value directly, go manually. (Ex. if date is not settable directly, click on the calendar icon and set the date)
"""

        # Make sure to use this pattern otherwise the exiting rules will be lost
        return f'{existing_rules}\n{new_rules}'



agent = Agent(
    task="Fill all fields(text, int, upload and date fields) in the job application using provided data",
    initial_actions=initial_actions,
    controller=controller,
    llm=llm,   
    browser=browser,
    retry_delay=20,
    max_actions_per_step=15,
    sensitive_data=sensitive_data,
    available_file_paths=["/home/bhanu/projects/ai-playground/browser/tmp.txt"],
    system_prompt_class=MySystemPrompt,
    generate_gif=True

)

    
# Basic configuration
config = BrowserConfig(
    headless=True,
    disable_security=True
)

browser = Browser(config=config)
async def main():
    result = await agent.run()
    return result


In [8]:
res = await main()

INFO     [agent] 🚀 Starting task: Fill all fields(text, int, upload and date fields) in the job application using provided data
INFO     [controller] 🔗  Opened new tab with http://localhost:8001/static/index.html
INFO     [agent] 📍 Step 1
INFO     [agent] 👍 Eval: Success - Opened the job application page
INFO     [agent] 🧠 Memory: Filling the job application form. 0 out of 7 text fields filled
INFO     [agent] 🎯 Next goal: Fill the text fields in the form
INFO     [agent] 🛠️  Action 1/6: {"input_text":{"index":0,"text":"<secret>first_name</secret> <secret>last_name</secret>"}}
INFO     [agent] 🛠️  Action 2/6: {"input_text":{"index":1,"text":"<secret>email</secret>"}}
INFO     [agent] 🛠️  Action 3/6: {"input_text":{"index":2,"text":"<secret>phone</secret>"}}
INFO     [agent] 🛠️  Action 4/6: {"input_text":{"index":3,"text":"Software Engineer"}}
INFO     [agent] 🛠️  Action 5/6: {"input_text":{"index":5,"text":"<secret>desired_salary</secret>"}}
INFO     [agent] 🛠️  Action 6/6: {"input_tex